In [1]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from utils.customloader import CustomDataset, DatasetSplit
from utils.separate_into_classes import separate_into_classes
#from utils.arguments import Args
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar
from models.Fed import FedAvg
from models.test import test_img
from utils.smooth_crossentropy import SmoothCrossEntropyLoss
from utils.dataloader import get_dataloader, set_seed
from utils.train_glob import train_global_model, test_model
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import copy
from sklearn.model_selection import train_test_split



import torch
class Args:
    #federated arugments
    epochs=10
    num_users=10
    local_ep=3
    local_bs=10
    bs=128
    lr=0.01
    momentum=0.5
    split='user'
    
    
    #model arguments
    model='mnist'
    kernel_num=9
    kernel_sizes='3,4,5'
    norm='batch_norm'
    num_filters=32
    max_pool=True
    
    #other arguments
    #data='mnist'
    #iid='store_true'
    num_channels=1
    num_classes=10
    #stopping_rounds=10
    verbose='store_true'
    seed=1
    #all_clients='store_true'
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    
    

args = Args()    
##############SET SEEDS FOR REPRODUCIBILITY#############
np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
# if you are suing GPU
torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)


torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True    
##############~SET SEEDS FOR REPRODUCIBILITY#############

## Define Dataloader /  Model / Optimizer / Loss

In [2]:
transform = transforms.Compose([ transforms.ToTensor(),  transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('./data/mnist/', train=True, download=True, transform=transform)
dataset_test = datasets.MNIST('./data/mnist/', train=False, download=True, transform=transform)


global_train_loader = DataLoader(dataset_train, batch_size=args.bs, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset_test, batch_size=args.bs, shuffle=False, num_workers=0)

## Define model 

In [3]:
#Before fedlearning
path_checkpoint='./test'

net_glob = CNNMnist(args=args).to(args.device)
net_glob.train()
sloss = F.cross_entropy

## Define Distribution of Data

In [4]:
dict_users = mnist_iid(global_train_loader.dataset.data, args.num_users)
len(dict_users[1])

6000

In [5]:
unique, counts = np.unique(global_train_loader.dataset.targets, return_counts=True)
print(unique)
print(counts)
sorted_y = copy.deepcopy(global_train_loader.dataset.targets)
sorted_index_y = np.argsort(np.squeeze(sorted_y))

class_dist=[]

for i in range(args.num_classes):
    print(i)
    class_dist.append(np.array(sorted_index_y[sum(counts[:i]):sum(counts[:i+1])], dtype=np.int64))
    
non_iid = np.array(class_dist)

[0 1 2 3 4 5 6 7 8 9]
[5923 6742 5958 6131 5842 5421 5918 6265 5851 5949]
0
1
2
3
4
5
6
7
8
9


In [6]:
individual = []
for j in range(10):
    individual.append(np.array_split(class_dist[j], 10))

user_dist=[]
for i in range(10):
    temp=[]
    for j in range(10):
        temp.append(individual[j][i])
        
    
    user_dist.append((np.concatenate(temp)).astype(np.int64))    
    
iid=np.array(user_dist)

In [7]:
def mnist_noniid(dataset, num_users):
    """
    Sample non-I.I.D client data from MNIST dataset
    :param dataset:
    :param num_users:
    :return:
    """
    num_shards, num_imgs = 200, 300
    idx_shard = [i for i in range(num_shards)]
    dict_users = {i: np.array([], dtype='int64') for i in range(num_users)}
    idxs = np.arange(num_shards*num_imgs)
    labels = dataset.train_labels.numpy()

    # sort labels
    idxs_labels = np.vstack((idxs, labels))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]
    idxs = idxs_labels[0,:]

    # divide and assign
    for i in range(num_users):
        rand_set = set(np.random.choice(idx_shard, 2, replace=False))
        idx_shard = list(set(idx_shard) - rand_set)
        for rand in rand_set:
            dict_users[i] = np.concatenate((dict_users[i], idxs[rand*num_imgs:(rand+1)*num_imgs]), axis=0)
    return dict_users


In [8]:
non_iid_again = mnist_noniid(global_train_loader.dataset, args.num_users)

/home/imtl/.conda/envs/imtl/lib/python3.8/site-packages/torchvision/datasets/mnist.py:45: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [9]:
cur_data_set = non_iid

# 1. Train Global Model

In [10]:
from utils.multiprocessing import work, multi_train_local_dif
from torch.utils.data.sampler import Sampler
from torchvision import datasets, transforms
import torch.multiprocessing as mp

## a. Run FedLearning in parallel

In [11]:
#Select percentage of participating clients
frac=1.0

In [ ]:
sloss2 = F.cross_entropy
global_model = copy.deepcopy(net_glob)

for i in range(args.epochs):

    m = max(int(frac * args.num_users), 1)

    print('--------------------------------------------')
    print("\n\n\nstart training epoch : " + str(i) + "\nNum_users:  " + str(m) + "\n\n\n")

    print('--------------------------------------------')

    procs=[]
    loss_locals=[]
    w_locals=[]        

    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:

        local = LocalUpdate(args=args, user_num=idx, loss_func=sloss, dataset=global_train_loader.dataset, idxs=cur_data_set[idx])
        w, loss = local.train(net=copy.deepcopy(global_model).to(args.device))

        w_locals.append(w)
        loss_locals.append(loss)

    print('--------------------------------------------\n\n')
    w_glob = FedAvg(w_locals)
    global_model.load_state_dict(w_glob)
    test_model(global_model, test_loader, sloss)
    print('\n\n--------------------------------------------')






## b. Test Model

In [13]:
#After fedlearning
print('Before Clustering Learning -- checkpoint')
test_model(global_model, test_loader, sloss)

Before Clustering Learning -- checkpoint

Test set: Average loss: 0.01570 
Accuracy: 3394/10000 (33.94%)



In [18]:
!git config user.name "sonhamin"
!git config user.email "sonhamin3@gmail.com"

In [8]:
!git remote add origin https://github.com/sonhamin/fedlearning_mod

In [14]:
!git add ./FedLearning.ipynb
!git add ./utils/*
!git add ./models/*


In [19]:
!git commit -m "add files"

[master (root-commit) 5ccb8c7] add files
 9 files changed, 783 insertions(+)
 create mode 100644 FedLearning.ipynb
 create mode 100644 models/Fed.py
 create mode 100644 models/Update.py
 create mode 100644 models/__init__.py
 create mode 100644 utils/__init__.py
 create mode 100644 utils/customloader.py
 create mode 100644 utils/dataloader.py
 create mode 100644 utils/multiprocessing.py
 create mode 100644 utils/train_glob.py


In [20]:
!git push -u origin master

Username for 'https://github.com': ^C


In [11]:
!ls

FedLearning.ipynb  data  models  utils
